In [52]:
import tensorflow as tf
import pandas as pd
import numpy as np
import string
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.model_selection import train_test_split
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dense, Input, Dropout, Conv1D, Flatten, GlobalMaxPool1D, Reshape
from nltk.corpus import stopwords
from collections import defaultdict
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from bert import tokenization

In [2]:
trainCSV = pd.read_csv('./forHiper')

In [3]:
x = trainCSV.drop(['id', 'text', 'keyword', 'location','target'], axis=1)
y = trainCSV.pop('target')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.01, random_state=42)

## Busco hiperparametros

GridSearch para batch_size, epochs y optimizador.

In [5]:
def build_model(optimizer):
  model = Sequential()
  model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

In [6]:
parameters = parameters = {'batch_size': [8,16,32],
             'epochs':[5,10,50,100,200,500],
             'optimizer': ['adadelta', 'rmsprop', 'adam']}

In [ ]:
estimator = KerasClassifier(build_fn=build_model, verbose=0)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10)
# grid_search.fit(X_train, y_train)
# grid_search.best_params_

GridSearch para cantidad de neuronas por capa.

In [ ]:
#Capas
def build_model(l1, l2):
  model = Sequential()
  model.add(Dense(l1, input_shape=(X_train.shape[1],), activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(l2, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
parameters = parameters = {'l1':[8,16,32,64,128,256],
                           'l2':[8,16,32,64,128,256]}

In [ ]:
estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=10)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10)
# grid_search.fit(X_train, y_train)
# grid_search.best_params_

GridSearch para valor de dropout.

In [ ]:
#Dropouts
def build_model(d1, d2):
    model = Sequential()
    model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(d1))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(d2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
parameters = {'d1':[0.01,0.1,0.2,0.25],
              'd2':[0.01,0.1,0.2,0.25]}

In [ ]:
estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=200)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10)
# grid_search.fit(X_train, y_train)
# grid_search.best_params_

## Entrenamiento de modelo

In [41]:
def get_compiled_model():
# (64, 16), (64, 32), (128, 16) va bien
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu')) 
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # adadelta
    return model

In [42]:
model = get_compiled_model()
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 64)                24256     
_________________________________________________________________
dropout_27 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 16)                1040      
_________________________________________________________________
dropout_28 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 17        
Total params: 25,313
Trainable params: 25,313
Non-trainable params: 0
_________________________________________________________________


In [43]:
# model.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
471/471 [==============================] - 0s 735us/step - loss: 0.7142 - accuracy: 0.6364
Epoch 2/10
471/471 [==============================] - 0s 811us/step - loss: 0.5087 - accuracy: 0.7600
Epoch 3/10
471/471 [==============================] - 0s 800us/step - loss: 0.4677 - accuracy: 0.7938
Epoch 4/10
471/471 [==============================] - 0s 756us/step - loss: 0.4503 - accuracy: 0.7994
Epoch 5/10
471/471 [==============================] - 0s 799us/step - loss: 0.4358 - accuracy: 0.8063
Epoch 6/10
471/471 [==============================] - 0s 766us/step - loss: 0.4290 - accuracy: 0.8100
Epoch 7/10
471/471 [==============================] - 0s 777us/step - loss: 0.4139 - accuracy: 0.8173
Epoch 8/10
471/471 [==============================] - 0s 792us/step - loss: 0.4143 - accuracy: 0.8170
Epoch 9/10
471/471 [==============================] - 0s 771us/step - loss: 0.4040 - accuracy: 0.8248
Epoch 10/10
471/471 [==============================] - 0s 771us/step - loss: 0.394

In [45]:
# results = model.evaluate(X_test, y_test, batch_size=16)
# print("test loss, test acc:", results)

5/5 [==============================] - 0s 918us/step - loss: 0.4592 - accuracy: 0.8182
test loss, test acc: [0.45919644832611084, 0.8181818127632141]


In [46]:
# testCSV = pd.read_csv('./testWithFeatures.csv')
# x_predict = testCSV.drop(['id', 'text', 'keyword', 'location'], axis=1)
# x_predict.head()

,tweet_length,users_tagged,hashtags,links,tweet_size,Valid_location,Total_elements,links_hash,links_users,hash_users,...,291,292,293,294,295,296,297,298,299,keywordAppearance
0,34,0,0,0,0,0,0,0,0,0,...,0.066162,-0.117676,0.805176,-0.536133,-0.091187,-0.384521,-0.329102,0.099121,-0.140366,0
1,64,0,1,0,5,0,1,1,0,1,...,0.013672,-0.553955,0.526291,0.255127,0.164307,-0.295898,-0.418701,-0.092163,-0.280029,0
2,96,0,0,0,5,0,0,0,0,0,...,-0.291565,-1.075439,0.923950,0.147629,0.764648,-0.166138,-0.417114,0.687012,0.234161,0
3,40,0,2,0,0,0,2,2,0,2,...,-0.357422,0.023369,0.978516,0.838867,0.009521,-0.522461,-0.157227,0.027344,0.166504,0
4,45,0,0,0,0,0,0,0,0,0,...,-0.029785,-0.608398,0.401367,-0.157715,0.350220,-0.752441,0.451477,0.156250,0.187134,0


In [48]:
# predictions = model.predict_classes(x_predict)

In [49]:
# b = predictions.ravel()
# pd.Series(b).value_counts()

0    3263
dtype: int64

In [50]:
# submissionDf = pd.read_csv('./sample_submission.csv')
# submissionDf['target'] = b

In [51]:
# submissionDf.head(10)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
5,12,0
6,21,0
7,22,0
8,27,0
9,29,0


In [ ]:
submissionDf.to_csv('submission.csv', index = False)

## BERT encoding

In [ ]:
# !wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [57]:
tf.gfile = tf.io.gfile

In [59]:
# %%time
# module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
# bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 10.1 s, sys: 1.63 s, total: 11.8 s
Wall time: 12.2 s


In [54]:
# train = pd.read_csv('./train.csv', dtype={'id': np.int16, 'target': np.int8})
# test = pd.read_csv('./test.csv', dtype={'id': np.int16})

In [55]:
# test['text'] = test['text'].fillna('')

In [56]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts: # tokeniza cada tweet
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2] # corta a max len y deja espacio para los token especiales
        input_sequence = ["[CLS]"] + text + ["[SEP]"] #Agrego tokens especiales
        pad_len = max_len - len(input_sequence) # calculo padding
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) #convierto tokens a ids
        tokens += [0] * pad_len #sumo padding
        pad_masks = [1] * len(input_sequence) + [0] * pad_len #creo la mask que me dice que es padding 
        segment_ids = [0] * max_len # es para tener distintas secuencias, lo dejamos en 0 porque todo es un tweet
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [60]:
# vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
# tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
# train_input = bert_encode(train.text.values, tokenizer, max_len=160)
# test_input = bert_encode(test.text.values, tokenizer, max_len=160)
# train_labels = train.target.values

In [ ]:
# train_input

## Defino model

In [61]:
def build_model1(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def build_model2(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out_first_layer    = Dense(16, activation='relu')(clf_output)
    out_first_dropout  = Dropout(0.2)(out_first_layer)
    out_second_layer   = Dense(8, activation='relu')(out_first_dropout)
    out_second_dropout = Dropout(0.2)(out_second_layer)
    out = Dense(1, activation='sigmoid')(out_second_dropout)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def build_model3(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out_first_layer    = Dense(128, activation='relu')(clf_output)
    out_first_dropout  = Dropout(0.2)(out_first_layer)
    out_second_layer   = Dense(64, activation='relu')(out_first_dropout)
    out_second_dropout = Dropout(0.2)(out_second_layer)
    out = Dense(1, activation='sigmoid')(out_second_dropout)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def build_model4(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    print(sequence_output.shape)
    clf_output = sequence_output[:, 0, :]
    reshape            = Reshape((1024,1))(clf_output)
    out_first_filter   = Conv1D(32, kernel_size=3, activation='relu', input_shape=(1024,1))(reshape)
    out_second_filter  = Conv1D(64, kernel_size=3, activation='relu')(out_first_filter)
    out_max_pooling    = GlobalMaxPool1D()(out_second_filter)
    out_flatten        = Flatten()(out_max_pooling)
    out_first_layer    = Dense(16, activation='relu')(out_flatten)
    out_second_dropout = Dropout(0.2)(out_first_layer)
    # out_second_layer   = Dense(8, activation='relu')(out_second_dropout)
    # out_third_dropout  = Dropout(0.2)(out_second_layer)
    out = Dense(1, activation='sigmoid')(out_second_dropout)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Entrenamiento

In [62]:
# model = build_model2(bert_layer, max_len=160) # best_score model! 
# model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [ ]:
# checkpoint = ModelCheckpoint('model_train.h5', monitor='val_loss', save_best_only=True)

# train_history = model.fit(
#     train_input, train_labels,
#     validation_split=0.2,
#     epochs=3,
#     callbacks=[checkpoint],
#     batch_size=16
# )

In [ ]:
# model.load_weights('modelv_train.h5')

In [ ]:
# test_pred = model.predict(test_input)

In [ ]:
# submission = pd.read_csv('./sample_submission.csv')
# submission.head()

In [ ]:
# submission['target'] = test_pred.round().astype(int)
# submission.to_csv('submission.csv', index=False)